# パラメータの最適化

In [13]:
%matplotlib inline

In [14]:
#コード準備
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [15]:
#ハイパーパラメータ
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [16]:
#最適化ループ(ハイパーパラメータを設定後、訓練で最適化のループを回すことで、モデルを最適化)

loss_fn = nn.CrossEntropyLoss() #loss function（損失関数）の初期化、定義

In [17]:
#最適化手法設定(確率的勾配降下法：Stochastic Gradient Descent)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

訓練ループ内で、最適化（optimization）は3つのステップから構成される。

[1] ``optimizer.zero_grad()``を実行し、モデルパラメータの勾配をリセットする。

勾配の計算は蓄積されていくので、毎イテレーション、明示的にリセットする。

<br>

[2] 続いて、``loss.backwards()``を実行し、バックプロパゲーションを実行する。

PyTorchは損失に対する各パラメータの偏微分の値（勾配）を求める。

<br>

[3] 最後に、``optimizer.step()``を実行し、各パラメータの勾配を使用してパラメータの値を調整する。

In [ ]:
#最適化を実行するコードをループするtrain_loopの定義
def train_loop(dataloader, model, loss_fn, optimaizer):
    size = len(dataloader.dataset) #len(dataloader) だと「バッチ数」
                                   #len(dataloader.dataset)だと「サンプル数」
    for batch, (X, y) in enumerate(dataloader): #batch=0,1,2,3・・・（batch１個につきデータ６４個）
        #予測と損失の計算
        pred = model(X)
        loss = loss_fn(pred, y) #損失（関数）の値（要素１つ）

        #バックプロパゲーション（誤差逆伝播法）
        optimizer.zero_grad() #前回の勾配をリセット
        loss.backward() #損失の勾配を計算（誤差逆伝播）
        optimizer.step() #勾配を使ってパラメータを更新

        #学習中に、進捗と損失を一定間隔で表示する処理
        if batch % 100 == 0: #100バッチごとにログを表示する
            loss =loss.item() #ログや表示用に使うだけなら、勾配情報は不要なので数値に変換して計算グラフから切り離すのが安全
            current = batch * len(X) #len(X) → 1つのバッチに入っているサンプル数（バッチサイズ（今回だと６４））
                                     #つまり、currentは今まで処理したサンプル数
            print(f"loss: {loss:.7f} [{current:>5d}/{size:>5d}]")

#テストデータに対してモデルの性能を評価するtest_loopの定義
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader: #(X,y)が６４個ずつ返ってくる
            pred = model(X) #pred は形が [batch_size, クラス数] のテンソル
            test_loss += loss_fn(pred, y).item()
            correct += (
                (pred.argmax(1) == y) #True/False の[batch_size] の整数テンソル（配列）
                .type(torch.float) #boolを1.0/0.0にして数値として足せるようにするため
                .sum() #sum()はテンソルの全要素（の数値）を足し合わせる関数
                .item()
            )

    test_loss /= size
    correct /=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")        

`enumerate` は **「繰り返し処理に、要素と同時にインデックス番号（カウンター）も付けてくれる関数」** 。
`in dataloader` との違いは、**インデックス番号を取るかどうか**。

---

## 1. 基本的な動き

普通の `for ... in ...` は要素だけを返す。

```python
fruits = ["apple", "banana", "orange"]

for fruit in fruits:
    print(fruit)
```

出力：

```
apple
banana
orange
```

インデックス番号（0, 1, 2…）は自分で数えるしかない。

---

`enumerate` を使うと、**インデックス番号と要素のペア**が返ってくる。

```python
for index, fruit in enumerate(fruits):
    print(index, fruit)
```

出力：

```
0 apple
1 banana
2 orange
```

---

## 2. DataLoaderの場合

`DataLoader` はミニバッチごとに `(X, y)` を返す。

* `X` → 入力データ（画像など）
* `y` → 正解ラベル

普通にループすると：

```python
for (X, y) in dataloader:
    # Xとyだけ取れる（バッチ番号はない）
```

`enumerate` を使うと：

```python
for batch, (X, y) in enumerate(dataloader):
    # batch → 0, 1, 2,... （ミニバッチ番号）
    # X, y  → そのバッチのデータ
```

これで「何番目のバッチか」を簡単に知ることができる。

---

## 3. 違いのまとめ

| 書き方                                          | 返ってくる値         | 使いどころ           |
| -------------------------------------------- | -------------- | --------------- |
| `for X, y in dataloader`                     | データだけ          | バッチ番号が不要なとき     |
| `for batch, (X, y) in enumerate(dataloader)` | `(バッチ番号, データ)` | ログ表示や進捗管理をしたいとき |

---

## 4. 補足：`enumerate`の第2引数

`enumerate` はカウンターの開始値も指定できる。

```python
for batch, (X, y) in enumerate(dataloader, start=1):
    print(batch)  # 1からスタート
```

---


## なぜtype()でテンソルの要素の型を変えられるのか

確かに通常は **`dtype`** を使って型を変えるのが一般的ですが、
PyTorch では `.type()` メソッドでも型変換ができます。

---

## 1. `.type()` と `.to()` と `.dtype` の違い

### `.type(dtype)`

* PyTorch の **Tensorクラスのメソッド**
* 引数に `torch.float` や `torch.int64` のような型クラスを渡すと、その型の新しいテンソルを返す

```python
x = torch.tensor([True, False])
x_f = x.type(torch.float)  # torch.float32 に変換
print(x_f.dtype)  # torch.float32
```

---

### `.to(dtype)`

* より汎用的な変換メソッド
* デバイス（CPU/GPU）や型をまとめて変えられる

```python
x_f = x.to(torch.float)  # dtypeだけ変更
x_gpu = x.to("cuda")     # GPUに移動
```

---

### `.dtype`

* これは **属性** なので、型を取得するだけ（型は変わらない）

```python
print(x.dtype)   # torch.bool
x.dtype = torch.float   # ❌ 書き換えはできない
```

---

## 2. なぜ `(pred.argmax(1) == y).type(torch.float)` で型が変わるのか

* `(pred.argmax(1) == y)` の結果は **torch.bool型テンソル**
* `.type(torch.float)` によって、新しいテンソルが作られ、全要素が True→1.0 / False→0.0 に変換される
* これは内部で\*\*型キャスト（type casting）\*\*が行われるから

---

## 3. 注意点

* 新しいPyTorchコードでは `.type()` より `.to(dtype=torch.float)` や `.float()` を使う方が推奨されます

```python
(pred.argmax(1) == y).float()
# 同じ意味で型変換（簡潔）
```

---

つまり `.type()` でも `dtype` が変わるのは、
**「型クラスを引数に取る型変換メソッド」だから** です。
`dtype` 属性は単なる「型の情報」で、変換ではなく取得専用です。


## Python の文字列フォーマット指定
`>0.1f` は **Python の文字列フォーマット指定** の一部で、
`0.1f` が「浮動小数点数を小数第1位まで表示」という意味である。

具体的にはこうなる：

* **`f`** → float形式（小数点付きの10進数）で表示
* **`.1`** → 小数点以下1桁に丸める
* **`0`** → 桁数指定のときにゼロ埋めする（ただし `0.1f` の場合は幅指定が無いので影響なし）

例えば：

```python
x = 3.14159
print(f"{x:0.1f}")  # -> '3.1'
print(f"{x:0.3f}")  # -> '3.142'
```

もし `>` の部分も含まれているなら、それは**整列指定**。

* **`>`** → 右寄せ（指定幅に合わせて右側に置く）

例：

```python
x = 3.14159
print(f"{x:>6.1f}")  # 幅6で右寄せ、小数1桁
# 出力: '   3.1'  (前に半角スペース3つ)
```

つまり **`>0.1f`** は「右寄せ＋小数点以下1桁＋ゼロ埋め表示」の条件を組み合わせたフォーマットになる。

In [19]:
#再定義（しなくてもよい）
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.2974241 [    0/60000]
loss: 2.2970819 [ 6400/60000]
loss: 2.2742522 [12800/60000]
loss: 2.2711635 [19200/60000]
loss: 2.2713902 [25600/60000]
loss: 2.2348692 [32000/60000]
loss: 2.2607467 [38400/60000]
loss: 2.2233007 [44800/60000]
loss: 2.2340639 [51200/60000]
loss: 2.2143164 [57600/60000]
Test Error: 
 Accuracy: 40.7%, Avg loss: 0.034630 

Epoch 2
-------------------------------
loss: 2.2164435 [    0/60000]
loss: 2.2172394 [ 6400/60000]
loss: 2.1476340 [12800/60000]
loss: 2.1610136 [19200/60000]
loss: 2.1612842 [25600/60000]
loss: 2.0922568 [32000/60000]
loss: 2.1457071 [38400/60000]
loss: 2.0727930 [44800/60000]
loss: 2.1037798 [51200/60000]
loss: 2.0362027 [57600/60000]
Test Error: 
 Accuracy: 45.3%, Avg loss: 0.031986 

Epoch 3
-------------------------------
loss: 2.0770097 [    0/60000]
loss: 2.0648038 [ 6400/60000]
loss: 1.9247109 [12800/60000]
loss: 1.9478282 [19200/60000]
loss: 1.9693801 [25600/60000]
loss: 1.8537853 [32000/600